# 🎯 Exemplos de Outros Tipos de Triggers

Este notebook apresenta exemplos de configuração para os outros tipos de triggers disponíveis no Lakeflow Jobs.

## 📋 Tipos de Triggers Disponíveis

1. ✅ **Time-based (Agendado)** - Executa em horários definidos
2. ✅ **Continuous (Contínuo)** - Execução constante, sempre ativa
3. ✅ **File Arrival** - Dispara quando novo arquivo é detectado (ver notebook 05)
4. ✅ **Manual** - Iniciado pelo usuário
5. ✅ **Table Update** - Executa quando há alteração em tabela (ver notebook 06)


## ⏰ Trigger 1: Time-based (Agendado)

### Quando usar:
- Processamento batch diário, semanal ou mensal
- Relatórios agendados
- Limpeza de dados periódica
- Backup de tabelas

### Configuração:

**Trigger Type:** `Scheduled`

**Schedule:**
- **Cron expression:** `0 0 2 * * ?` (todos os dias às 02:00)
- **Timezone:** `America/Sao_Paulo`

**Exemplos de Cron:**
- `0 0 2 * * ?` - Diário às 02:00
- `0 0 0 ? * MON` - Toda segunda-feira à meia-noite
- `0 0 0 1 * ?` - Primeiro dia de cada mês
- `0 */30 * * * ?` - A cada 30 minutos

### Exemplo de Job:

**Nome:** `daily_claims_ingestion`

**Schedule:** `0 0 2 * * ?` (diário às 02:00)

**Task:** Executa `ingestion_claim.py` para reprocessar todos os dados


## 🔄 Trigger 2: Continuous (Contínuo)

### Quando usar:
- Processamento de streaming em tempo real
- Monitoramento contínuo de dados
- Pipelines que precisam estar sempre ativos
- Processamento de eventos em tempo real

### Configuração:

**Trigger Type:** `Continuous`

**Restart on failure:**
- ✅ Marque para reiniciar automaticamente em caso de falha

**Max concurrent runs:**
- Exemplo: `1` (apenas uma execução por vez)
- Evita sobrecarga do cluster

### Exemplo de Job:

**Nome:** `continuous_claims_streaming`

**Trigger:** Continuous

**Task:** Processa dados de streaming usando `readStream()` do Spark

**⚠️ Atenção:** Jobs contínuos consomem recursos constantemente. Use apenas quando necessário.


## 👤 Trigger 3: Manual

### Quando usar:
- Testes e desenvolvimento
- Execuções sob demanda
- Reprocessamento manual de dados
- Validação de pipelines

### Configuração:

**Trigger Type:** `Manual`

- Não requer configuração adicional
- Job só executa quando iniciado manualmente pelo usuário

### Como executar:

1. Acesse **Lakeflow Jobs** > Seu Job
2. Clique em **Run now**
3. Ou use a API do Databricks para executar programaticamente

### Exemplo de Job:

**Nome:** `manual_data_validation`

**Trigger:** Manual

**Task:** Executa validações e testes de qualidade de dados

**Uso:** Executar manualmente após mudanças no código ou schema


## 📊 Comparação de Triggers

| Trigger | Uso Ideal | Frequência | Recursos |
|---------|-----------|------------|----------|
| **Time-based** | Processamento batch agendado | Periódico (horário fixo) | Baixo (executa apenas no horário) |
| **Continuous** | Streaming em tempo real | Constante | Alto (sempre ativo) |
| **File Arrival** | Ingestão quando arquivo chega | Event-driven | Médio (só quando há arquivo) |
| **Table Update** | Pipeline reativo a mudanças | Event-driven | Médio (só quando há mudança) |
| **Manual** | Testes e execuções sob demanda | Sob demanda | Baixo (apenas quando executado) |

## 🎯 Recomendações por Cenário

### Ingestão de Arquivos CSV
- ✅ **File Arrival** - Ideal para processar arquivos assim que chegam

### Transformação Bronze → Silver
- ✅ **Table Update** - Ideal para manter dados silver sincronizados

### Relatórios Diários
- ✅ **Time-based** - Executa em horário fixo todos os dias

### Streaming de Dados
- ✅ **Continuous** - Processa dados em tempo real continuamente

### Desenvolvimento e Testes
- ✅ **Manual** - Permite controle total sobre quando executar


## 🔗 Combinando Múltiplos Triggers

Você pode configurar **múltiplos triggers** para o mesmo job:

### Exemplo: Job com 2 Triggers

**Job:** `flexible_claims_processing`

**Trigger 1:** File Arrival
- Monitora: `/Volumes/.../claims*.csv`
- Processa automaticamente quando arquivo chega

**Trigger 2:** Time-based
- Schedule: `0 0 1 * * ?` (diário às 01:00)
- Reprocessa todos os dados diariamente

**Resultado:** 
- Processamento automático quando arquivo chega (File Arrival)
- Reprocessamento completo diário (Time-based)
- Garante que dados estejam sempre atualizados


In [ ]:
# Exemplo: Criar job programaticamente usando Databricks API
# Este código demonstra como criar um job com trigger via API

import json

# Configuração do job com trigger Time-based
job_config = {
    "name": "daily_claims_ingestion",
    "tasks": [
        {
            "task_key": "ingest_claims",
            "python_wheel_task": {
                "package_name": "ingestion_claim",
                "entry_point": "main"
            },
            "libraries": [
                {
                    "pypi": {
                        "package": "pyspark"
                    }
                }
            ],
            "python_wheel_task": {
                "parameters": [
                    "--catalog", "smart_claims_dev",
                    "--schema", "01_bronze"
                ]
            }
        }
    ],
    "schedule": {
        "quartz_cron_expression": "0 0 2 * * ?",
        "timezone_id": "America/Sao_Paulo",
        "pause_status": "UNPAUSED"
    },
    "max_concurrent_runs": 1,
    "timeout_seconds": 3600
}

print("📋 Exemplo de configuração de job com trigger Time-based:")
print(json.dumps(job_config, indent=2))

# Para criar o job, use a API do Databricks:
# POST /api/2.1/jobs/create
# Headers: Authorization: Bearer <token>
# Body: job_config (acima)
